In [160]:
# Covid-19 Impact on Economy jupyter notebook
# Author: Andy Juan
# Date: 2020-03-28
# Course: CS-GY 6513 Big Data
# Project: Course Project
import pandas as pd
import json

# Constants
twenty19_start_date = pd.to_datetime("01/01/2019")
twenty19_end_date = pd.to_datetime("12/31/2019")
twenty20_start_date = pd.to_datetime("01/01/2020")
twenty20_end_date = pd.to_datetime("12/31/2020")

### Confirmed Covid cases data ###
# Read csv file that has data for positive cases for 2020
# Specimen_date column will be used to represent confirmed positive cases.
dtypes = {'extract_date': 'object', 'specimen_date': 'object', 'Number_tested': 'int64',
          'Number_confirmed': 'int64', 'Number_hospitalized': 'int64', 'Number_deaths': 'int64'}
covid_data_df = pd.read_csv("covid_positive_and_deaths.csv", dtype=dtypes)

# Cast column type to pandas datetime type
# Replace invalid dates with NAT and drop all rows with NA values
covid_data_df['extract_date'] = pd.to_datetime(covid_data_df['extract_date'], errors='coerce')
covid_data_df['specimen_date'] = pd.to_datetime(covid_data_df['specimen_date'], errors='coerce')
covid_data_df = covid_data_df.dropna()

# Sort by date and filter for 2020
covid_data_df = covid_data_df.sort_values(by=['specimen_date'], ascending=True)
twenty20_df = covid_data_df[
    (covid_data_df["specimen_date"] >= twenty20_start_date) & 
    (covid_data_df["specimen_date"] <= twenty20_end_date)
]
# Group by date
twenty20_positive_df = twenty20_df.groupby(["specimen_date"], as_index=True).sum().reset_index()
#wenty20_positive_df.plot.line(x='specimen_date', y='Number_confirmed')

### SPY 2020 Closing prices ###
# Json file retrieved and stored locally from polygon.io. polygon rate limits queries.
spy_2020_json = None
with open("SPY_2020_daily_open_close.json") as f:
    spy_2020_json = json.load(f)
# Convert json data to pandas dataframe
spy_2020_df = pd.DataFrame(spy_2020_json["results"])
# Cast time type to pandas datetime
spy_2020_df['t'] = pd.to_datetime(spy_2020_df['t'], unit='ms', errors='coerce')
#sy_2020_df.plot.line(x='t', y='c')

### NYC legally operating businesses ###
nyc_business_df = pd.read_csv("Legally_Operating_Businesses.csv", low_memory=False)
# Cast date columsn to pandas datetime type
nyc_business_df['License Expiration Date'] = pd.to_datetime(nyc_business_df['License Expiration Date'], 
                                                              errors='coerce')
nyc_business_df['License Creation Date'] = pd.to_datetime(nyc_business_df['License Creation Date'], 
                                                            errors='coerce')
# Get the businesses that were created in 2019
nyc_19_new_bus_df = nyc_business_df[(nyc_business_df["License Creation Date"] >= twenty19_start_date) &
                                    (nyc_business_df["License Creation Date"] <= twenty19_end_date)
                                   ]
nyc_19_new_bus_df = nyc_19_new_bus_df.sort_values(by=['License Creation Date'], ascending=True)

# Get the businesses that were created in 2020
nyc_20_new_bus_df = nyc_business_df[(nyc_business_df["License Creation Date"] >= twenty20_start_date) &
                                    (nyc_business_df["License Creation Date"] <= twenty20_end_date)
                                   ]
nyc_20_new_bus_df= nyc_20_new_bus_df.sort_values(by=['License Creation Date'], ascending=True)
#print ("2019 count: " , len(nyc_19_new_bus_df))
#print ("20 count: ", len(nyc_20_new_bus_df))
#print ("Diff: ", len(nyc_19_new_bus_df) - len(nyc_20_new_bus_df))
#nyc_20_new_bus_df

2019 count:  9913
20 count:  3880
Diff:  6033


,DCA License Number,License Type,License Expiration Date,License Status,License Creation Date,Industry,Business Name,Business Name 2,Address Building,Address Street Name,...,Community Board,Council District,BIN,BBL,NTA,Census Tract,Detail,Longitude,Latitude,Location
254665,2093331-DCA,Business,2020-12-31,Inactive,2020-01-02,Electronics Store,DSN WIRELESS INC,NaN,8439,NEW UTRECHT AVE,...,311.0,47.0,3165305,3063320001,NaN,NaN,NaN,-74.001707,40.607953,"(40.60795285367902, -74.00170710129659)"
42874,2093323-2-DCA,Business,2022-12-31,Active,2020-01-02,Tobacco Retail Dealer,BHAKTI 2154 INC,842843940,2154,8TH AVE,...,110.0,9.0,1058363,1019220001,NaN,NaN,NaN,-73.955210,40.804623,"(40.80462288485693, -73.95520970235555)"
36765,2093299-DCA,Individual,2020-10-31,Inactive,2020-01-02,Tow Truck Driver,"MATABEEK, KISHORE R",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10229,2093322-2-DCA,Business,2020-11-30,Inactive,2020-01-02,Electronic Cigarette Dealer,KINGS APPLE FARMERS MARKET INC,NaN,1055,BRIGHTON BEACH AVE,...,313.0,48.0,3245420,3087090069,NaN,NaN,NaN,-73.957907,40.578288,"(40.578287599922426, -73.95790660257863)"
252142,2093316-2-DCA,Business,2020-12-31,Inactive,2020-01-02,Tobacco Retail Dealer,9TH GROCERY INC,NaN,350,9TH AVE,...,104.0,3.0,1068013,1007530078,NaN,NaN,NaN,-73.998181,40.750802,"(40.75080180263017, -73.99818096204716)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250731,2097374-DCA,Business,2021-12-31,Active,2020-12-31,Laundries,NEWASH INC,NaN,179,E 105TH ST,...,111.0,8.0,1051998,1016330033,NaN,NaN,Laundry Type: Industrial Laundry with Delivery,-73.945804,40.791172,"(40.79117242230543, -73.94580385400756)"
251050,2097371-DCA,Business,2023-03-31,Active,2020-12-31,Garage,301 EAST 40TH STREET PARKING CORP.,NaN,301,E 40TH ST,...,106.0,4.0,1038645,1013330001,NaN,NaN,"Vehicle Spaces: 108, Bicycle Spaces: 11",-73.973011,40.748493,"(40.74849306691836, -73.97301132022025)"
256438,2097370-DCA,Business,2023-02-28,Active,2020-12-31,Home Improvement Contractor,"INNOVATIVE RENOVATIONS, LLC",NaN,620,SHELDON AVE,...,503.0,51.0,5081601,5063380081,NaN,NaN,NaN,-74.195783,40.541337,"(40.54133699796005, -74.1957829149701)"
250771,2097372-DCA,Business,2023-03-31,Active,2020-12-31,Garage,115 EAST 87TH STREET PARKING CORP.,NaN,115,E 87TH ST,...,108.0,4.0,1048043,1015167502,NaN,NaN,"Vehicle Spaces: 198, Bicycle Spaces: 20",-73.955995,40.780563,"(40.78056287994857, -73.95599499048019)"
